## tech

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm

In [2]:
df = pd.read_json('../data/data.txt',lines=True,dtype={'E':'datetime64[ms]'},chunksize=200000)

In [3]:
name_dict = {
  "E": 'event_time',
  "s": 'symbol',
  "t": 'trade_id',
  "p": "price",     
  "q": "quantity",       
  "b": 'buyer_order_id',
  "a": 'seller_order_id', 
  "T": 'trade_time', 
  "m": 'is_marker_maker'
}

In [4]:

trade = pd.DataFrame()
for data in tqdm(df):

    tmp = data.query('e!="depthUpdate"').dropna(axis=1)
    tmp['type'] = tmp['p'].diff().where(lambda x: x!=0).fillna(method='ffill').mul(100).clip(-1,1).map({-1:'sell',1:'buy'})
    tmp=tmp.rename(columns=name_dict)
    trade = trade.append(tmp)
    if trade.shape[0] > 7e+5:
        break
        


2it [00:45, 22.82s/it]


In [6]:
trade.rename(columns=name_dict)

,e,event_time,symbol,buyer_order_id,seller_order_id,trade_id,price,quantity,trade_time,is_marker_maker,M,type
3,trade,2022-01-21 07:56:25.130,BTCUSDT,9063926759,9063926875,1.228137e+09,39118.31,0.00617,1.642752e+12,1.0,1.0,NaN
4,trade,2022-01-21 07:56:25.162,BTCUSDT,9063926878,9063926770,1.228137e+09,39118.32,0.00118,1.642752e+12,0.0,1.0,buy
6,trade,2022-01-21 07:56:25.268,BTCUSDT,9063926882,9063926770,1.228137e+09,39118.32,0.00430,1.642752e+12,0.0,1.0,buy
11,trade,2022-01-21 07:56:25.668,BTCUSDT,9063926915,9063926770,1.228137e+09,39118.32,0.00222,1.642752e+12,0.0,1.0,buy
13,trade,2022-01-21 07:56:25.745,BTCUSDT,9063926841,9063926924,1.228137e+09,39118.31,0.00261,1.642752e+12,1.0,1.0,sell
...,...,...,...,...,...,...,...,...,...,...,...,...
1499995,trade,2022-01-21 21:47:55.705,BTCUSDT,9074767705,9074767713,1.229140e+09,36786.85,0.03000,1.642802e+12,1.0,1.0,sell
1499996,trade,2022-01-21 21:47:55.706,BTCUSDT,9074767705,9074767714,1.229140e+09,36786.85,0.06809,1.642802e+12,1.0,1.0,sell
1499997,trade,2022-01-21 21:47:55.706,BTCUSDT,9074767680,9074767714,1.229140e+09,36785.01,0.01719,1.642802e+12,1.0,1.0,sell
1499998,trade,2022-01-21 21:47:55.709,BTCUSDT,9074767680,9074767716,1.229140e+09,36785.01,0.04000,1.642802e+12,1.0,1.0,sell


In [ ]:
du = pd.DataFrame()
for data in tqdm(df):
    tmp = data.query('e=="depthUpdate"').dropna(axis=1)

    tmp['b_first_q']=tmp['b'].explode().dropna().apply(lambda x: x[1] if x[1]!='0.00000000' else np.nan).groupby(level=0).first().astype(float)
    tmp['a_first_q']=tmp['a'].explode().dropna().apply(lambda x: x[1] if x[1]!='0.00000000' else np.nan).groupby(level=0).first().astype(float)

    tmp['b_first_p']=tmp['b'].explode().dropna().apply(lambda x: x[0] if x[1]!='0.00000000' else np.nan).groupby(level=0).first().astype(float)
    tmp['a_first_p']=tmp['a'].explode().dropna().apply(lambda x: x[0] if x[1]!='0.00000000' else np.nan).groupby(level=0).first().astype(float)
    tmp['is_seller_pres_bid']=tmp['b_first_q'].diff().apply(lambda x: x if x>0 else 0)
    tmp['is_buyer_pres_bid']=tmp[['b_first_q','b_first_p']].diff().apply(lambda x: abs(x['b_first_q']) *-1  if x['b_first_p'] <0 else 0,axis=1)

    tmp['is_seller_pres_ask']=tmp['a_first_q'].diff().apply(lambda x: x if x>0 else 0)
    tmp['is_buyer_pres_ask']=tmp[['a_first_q','a_first_p']].diff().apply(lambda x: abs(x['a_first_q']) *-1  if x['a_first_p'] <0 else 0,axis=1)
    tmp.drop(['e','u','U','b','a'],axis=1,inplace=True)
    du=du.append(tmp)
    print(du.memory_usage(deep=True).sum())
    if du.memory_usage(deep=True).sum() // 1000**2 > 300:
        break

### data transform

In [ ]:
n_ticks= 500
trade[f'ticks_{n_ticks}'] = np.arange(0,trade.shape[0]) // n_ticks
du[f'ticks_{n_ticks}'] = np.arange(0,du.shape[0]) // n_ticks
trade[f'ticks_{n_ticks}_dt'] = trade.groupby(f'ticks_{n_ticks}')['E'].transform('first')
du[f'ticks_{n_ticks}_dt'] = du.groupby(f'ticks_{n_ticks}')['E'].transform('first')


In [ ]:
n_ticks= 10
trade[f'ticks_{n_ticks}'] = np.arange(0,trade.shape[0]) // n_ticks
du[f'ticks_{n_ticks}'] = np.arange(0,du.shape[0]) // n_ticks
trade[f'ticks_{n_ticks}_dt'] = trade.groupby(f'ticks_{n_ticks}')['E'].transform('first')
du[f'ticks_{n_ticks}_dt'] = du.groupby(f'ticks_{n_ticks}')['E'].transform('first')

In [ ]:
ttl_pres_df = du.groupby('ticks_500_dt')[['is_buyer_pres_bid','is_seller_pres_bid']].sum().melt(ignore_index=False).reset_index()\
            .append(du.groupby('ticks_500_dt')[['is_buyer_pres_ask','is_seller_pres_ask']].sum().melt(ignore_index=False).reset_index())
ttl_pres_df['type']=ttl_pres_df['variable'].str.contains('bid').map({True:'bid',False:'ask'})

## Plots

### 1

In [ ]:
px.line(trade.groupby(pd.Grouper(key='E',freq='10S'))['t'].nunique(),title='Индикатор скорости сделок за 10 секунд').write_html('plots/1_speed.html')

In [ ]:
plot_1_df = pd.DataFrame()
for i in [10,30,60,120,500,1000,2000,3000,5000]:
    plot_1_df[i]=trade.groupby(pd.Grouper(key='E',freq=f'{i}S'))['t'].nunique()

In [ ]:
px.line(plot_1_df.melt(ignore_index=False).reset_index().dropna(),x='E',y='value',animation_frame='variable').write_html('plots/1_speed.html')

### 2

In [ ]:
px.line(trade.groupby([pd.Grouper(key='E',freq='10S')])['type'].value_counts().unstack(),title='Индикатор интенсивности продаж/покупок').write_html('plots/2_buysell_value_count.html')

In [ ]:
plot_2_df = pd.DataFrame()
for i in tqdm([10,30,60,120,500,1000,2000,3000,5000]):
    plot_2_df[i] = trade.groupby(pd.Grouper(key='E',freq=f'{i}S'))['type'].value_counts()
px.line(plot_2_df.melt(ignore_index=False).reset_index().dropna(),x='E',y='value',color='type',animation_frame='variable').write_html('plots/2_buysell_value_count.html')

### 3

In [ ]:
px.line(du.groupby(pd.Grouper(key='E',freq='10S'))['diff_p'].mean(),title='Индикатор спреда').write_html('plots/3_spread.html')

In [ ]:
plot_3_df = pd.DataFrame()
for i in tqdm([10,30,60,120,500,1000,2000,3000,5000]):
    plot_3_df[i] = du.groupby(pd.Grouper(key='E',freq=f'{i}S'))['diff_p'].mean()

In [ ]:
px.line(plot_3_df.melt(ignore_index=False).reset_index().dropna(),title='Индикатор спреда',x='E',y='value',animation_frame='variable').write_html('plots/3_spread.html')

In [ ]:
px.line(du.groupby(pd.Grouper(key='E',freq='10S'))[['b_first_p','a_first_p']].mean()\
    .join(trade.groupby(pd.Grouper(key='E',freq='10S'))['p'].last())\
        ,title='Линии поведение бида и аска относительно цены последней сделки').write_html('plots/3_bidask_price_comp.html')

### 4

In [ ]:
px.bar(du.groupby('ticks_500_dt')[['is_buyer_pres_bid','is_seller_pres_bid']].sum().melt(ignore_index=False).reset_index(),x='ticks_500_dt',y='value',color='variable',title='Чистые бары давлений').write_html('plots/4_raw_bar_pressure_bid.html')
px.bar(du.groupby('ticks_500_dt')[['is_buyer_pres_ask','is_seller_pres_ask']].sum().melt(ignore_index=False).reset_index(),x='ticks_500_dt',y='value',color='variable',title='Чистые бары давлений').write_html('plots/4_raw_bar_pressure_ask.html')

In [ ]:
px.bar(ttl_pres_df,x='ticks_500_dt',y='value',color='variable',facet_row='type',title='Чистые бары давлений').write_html('plots/4_raw_bar_pressure.html')

In [ ]:
px.bar(du.groupby('ticks_500_dt')[['is_buyer_pres_bid','is_seller_pres_bid']].sum().sum(axis=1),title='Разница давлений').write_html('plots/4_diff_bar_pressure_bid.html')
px.bar(du.groupby('ticks_500_dt')[['is_buyer_pres_ask','is_seller_pres_ask']].sum().sum(axis=1),title='Разница давлений').write_html('plots/4_diff_bar_pressure_ask.html')

In [ ]:
px.bar(ttl_pres_df.groupby(['ticks_500_dt','type'])['value'].sum().reset_index(),x='ticks_500_dt',y='value',facet_row='type',color='type',title='Разница давлений').write_html('plots/4_diff_bar_pressure.html')

### 5

In [ ]:
olhc = trade.groupby('ticks_10_dt')['p'].agg(['first','last','min','max'])
olhc_500 = trade.groupby('ticks_500_dt')['p'].agg(['first','last','min','max'])

In [ ]:
fig = go.Figure(data=go.Ohlc(x=olhc.index,open=olhc['first'],close=olhc['last'],low=olhc['min'],high=olhc['max']))
fig.write_html('plots/5_ohlc.html')

In [ ]:
fig = go.Figure(data=go.Ohlc(x=olhc_500.index,open=olhc_500['first'],close=olhc_500['last'],low=olhc_500['min'],high=olhc_500['max']))
fig.write_html('plots/5_ohlc_500.html')